## Note that:
The folder, 'DAIS2023CompetitionSubmission_Liu', includes 2 '.ipynb' files, a 'Datasets' folder, a 'BaselineModel_PointNet' folder and a final report in '.pdf'.

You should redirect this file to your working folder by edit the code, "os.chdir(...)".

Copy the test dataset into your working folder, i.e. '/DAIS2023CompetitionSubmission_Liu/Datasets/'.

Then start to run the whole file.

# **Load Testing Dataset**

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import proj3d
from mpl_toolkits.mplot3d import Axes3D

os.chdir('C:/Users/xl037/PycharmProjects/DAIS2023CompetitionSubmission_Liu/Datasets/')
cwd = os.getcwd()
# print(cwd)
test_data_Test_path = os.path.join(cwd, 'test_data.npy')
label_data_Test_path = os.path.join(cwd, 'test_label.npy')

test_data_Test = np.load(test_data_Test_path)
label_data_Test = np.load(label_data_Test_path)

print('The number of testing samples is', np.shape(test_data_Test)[0])

# **Load Training Dataset**

Here are the plots of all 3D point clouds from 190 samples. Green denotes category '0' and red denotes category '1'.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import proj3d
from mpl_toolkits.mplot3d import Axes3D


test_data_Train_path = os.path.join(cwd, 'train_data.npy')
label_data_Train_path = os.path.join(cwd, 'train_label.npy')

test_data = np.load(test_data_Train_path)
label_data = np.load(label_data_Train_path)

print('The number of training samples is', np.shape(test_data)[0])

index_0 = np.where(label_data == 0)[0]
index_1 = np.where(label_data == 1)[0]

# **Tensor-Voting based Segmentation for Reference Surface and Anomaly Surface**

Before using tensor voting method to calculate all the local features, we can use point cloud denoising method to denoise the whole point cloud data. We have prepared the denoising method, but it is not necessary here since that the current data are not noisy after our pre-checkings.

**Tensor Voting Method** for calculating node-wise physical features

In [ ]:
import numpy as np
import os
!pip install open3d==0.16
import open3d as o3d
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from collections import Counter


# First, we need to extract the local features of each node using tensor voting method
# For each node, find the k-nearest nodes and then calculate the L (Linearness), the P (Planarity), and the S (Sphericity)
def LocalFeatures(dataset_temp, num_nodes, radius):
  pcd = o3d.geometry.PointCloud()
  local_feature_temp = np.zeros((np.shape(dataset_temp)))
  pcd.points = o3d.utility.Vector3dVector(dataset_temp)
  kdtree = o3d.geometry.KDTreeFlann(pcd)
  for i in range(num_nodes):
    index_temp = i
    radius_temp = radius
    k = 0
    while k < 100:
      radius_temp = radius_temp*1.1
      k, idx, _ = kdtree.search_radius_vector_3d(pcd.points[index_temp], radius_temp)
      # print(k)
    neighbors = dataset_temp[idx, :]
    cov_temp = np.zeros((3, 3))
    mean = np.mean(neighbors, 0)
    for j in range(k):
      cov_temp += (neighbors[j, :] - mean.reshape((1, 3))).T @ (neighbors[j, :] - mean.reshape((1, 3))) / k
    # the eigen decomposition
    Lambda, U = np.linalg.eig(cov_temp)
    Lambda = list(Lambda)
    Lambda.sort(reverse=True)
    # print(Lambda)
    local_feature_temp[index_temp, 0] = Lambda[0] - Lambda[1]  # the L (Linearness)
    local_feature_temp[index_temp, 1] = Lambda[1] - Lambda[2]  # the P (Planarity)
    local_feature_temp[index_temp, 2] = Lambda[2]  # the S (Sphericity)
#     print(k)
  # print(local_feature_temp)
  return local_feature_temp

# calculate all the local features of all samples
local_features_all = []
radius_k = 0.1
for index in range(np.shape(test_data)[0]): local_features_all.append(LocalFeatures(test_data[index], np.shape(test_data[index])[0], radius_k))

# print(local_features_all[0])

For test dataset: ** Tensor Voting Method** for calculating node-wise physical features

In [ ]:
import numpy as np
import os
!pip install open3d==0.16
import open3d as o3d
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from collections import Counter


# First, we need to extract the local features of each node using tensor voting method
# For each node, find the k-nearest nodes and then calculate the L (Linearness), the P (Planarity), and the S (Sphericity)
def LocalFeatures(dataset_temp, num_nodes, radius):
  pcd = o3d.geometry.PointCloud()
  local_feature_temp = np.zeros((np.shape(dataset_temp)))
  pcd.points = o3d.utility.Vector3dVector(dataset_temp)
  kdtree = o3d.geometry.KDTreeFlann(pcd)
  for i in range(num_nodes):
    index_temp = i
    radius_temp = radius
    k = 0
    while k < 100:
      radius_temp = radius_temp*1.1
      k, idx, _ = kdtree.search_radius_vector_3d(pcd.points[index_temp], radius_temp)
      # print(k)
    neighbors = dataset_temp[idx, :]
    cov_temp = np.zeros((3, 3))
    mean = np.mean(neighbors, 0)
    for j in range(k):
      cov_temp += (neighbors[j, :] - mean.reshape((1, 3))).T @ (neighbors[j, :] - mean.reshape((1, 3))) / k
    # the eigen decomposition
    Lambda, U = np.linalg.eig(cov_temp)
    Lambda = list(Lambda)
    Lambda.sort(reverse=True)
    # print(Lambda)
    local_feature_temp[index_temp, 0] = Lambda[0] - Lambda[1]  # the L (Linearness)
    local_feature_temp[index_temp, 1] = Lambda[1] - Lambda[2]  # the P (Planarity)
    local_feature_temp[index_temp, 2] = Lambda[2]  # the S (Sphericity)
#     print(k)
  # print(local_feature_temp)
  return local_feature_temp

# calculate all the local features of all samples
local_features_all_Test = []
radius_k = 0.1
for index in range(np.shape(test_data_Test)[0]): local_features_all_Test.append(LocalFeatures(test_data_Test[index], np.shape(test_data_Test[index])[0], radius_k))

# print(local_features_all[0])

 **Local Features** (L, P, S) of training data:

In [ ]:
# The local features of all samples. It is noted that it is important to check the sensitivity of the searching parameters for selecting the K-nearest nodes
# class 0
f = plt.figure(figsize=(20, 4))
ax1 = f.add_subplot(131)
ax2 = f.add_subplot(132)
ax3 = f.add_subplot(133)
for index_sample in index_0:
    temp_feature = list(local_features_all[index_sample][:, 0:1])
    temp_feature.sort(reverse=True)
    ax1.plot(np.array(range(2048)), temp_feature, c='green')

    temp_feature = list(local_features_all[index_sample][:, 1:2])
    temp_feature.sort(reverse=True)
    ax2.plot(np.array(range(2048)), temp_feature, c='green')

    temp_feature = list(local_features_all[index_sample][:, 2:3])
    temp_feature.sort(reverse=True)
    ax3.plot(np.array(range(2048)), temp_feature, c='green')

# class 1
for index_sample in index_1:
    temp_feature = list(local_features_all[index_sample][:, 0:1])
    temp_feature.sort(reverse=True)
    ax1.plot(np.array(range(2048)), temp_feature, c='red')

    temp_feature = list(local_features_all[index_sample][:, 1:2])
    temp_feature.sort(reverse=True)
    ax2.plot(np.array(range(2048)), temp_feature, c='red')

    temp_feature = list(local_features_all[index_sample][:, 2:3])
    temp_feature.sort(reverse=True)
    ax3.plot(np.array(range(2048)), temp_feature, c='red')

# Set common labels
ax1.set_xlabel('Sorted point index')
ax1.set_ylabel('L (Linearness) value')
ax1.set_title('The Linearness')
# Set common labels
ax2.set_xlabel('Sorted point index')
ax2.set_ylabel('P (Planarity) value')
ax2.set_title('The Planarity')
# Set common labels
ax3.set_xlabel('Sorted point index')
ax3.set_ylabel('S (Sphericity) value')
ax3.set_title('The Sphericity')
f.suptitle('The local features for all samples (green color denotes class 0 and red color denotes class 1)', y=0.0001, fontsize=15)
f.show()

Discussion:

The node-wise physical features (including L, S, and P), which indicate physical information, can help us get physical-interpreted segmentations, such as the reference surface and the anomaly surface. If we directly apply clustering methods (such as k-means and DBSCAN) to the coordinates, the segmentations will lack the physical interpretations, e.g., the reference planes and anomalies. Moreover, the direct clustering of the coordinates is not stable or robust, when the shape or the junction saliency is changed. Based on the node-wise physical features (including L, S, and P), we can incorporate different kinds of clustering methods to achieve physical-interpreted features for classification purposes.

**Gaussian Mixture Model** for selecting reference surface nodes

In [ ]:
import numpy as np
import os
# !pip install open3d==0.16
import open3d as o3d
from sklearn import mixture
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from collections import Counter
import scipy.optimize as optimize
import matplotlib.pyplot as plt


def perp_error(params, xyz):
    a, b, c, d = params
    x, y, z = xyz
    length = np.sqrt(a**2 + b**2 + c**2)
    return (np.abs(a * x + b * y + c * z + d) / length).mean()


def SVD(X):
    # Find the average of points (centroid) along the columns
    C = np.average(X, axis=0)
    # Create CX vector (centroid to point) matrix
    CX = X - C
    # Singular value decomposition
    U, S, V = np.linalg.svd(CX)
    # The last row of V matrix indicate the eigenvectors of
    # smallest eigenvalues (singular values).
    N = V[-1]
    # Extract a, b, c, d coefficients.
    x0, y0, z0 = C
    a, b, c = N
    d = -(a * x0 + b * y0 + c * z0)
    return a, b, c, d

# os.chdir('/content/drive/MyDrive/DAISproject/Datasets')
# # Read point cloud:
# cwd = os.getcwd()
# test_data_path = os.path.join(cwd, 'train_data.npy')
# label_data_path = os.path.join(cwd, 'train_label.npy')

# test_data = np.load(test_data_path)
# label_data = np.load(label_data_path)

# change the index here to see different samples
index = 119

temp_data = local_features_all[index][:, :]

# Normalisation:
scaled_points = StandardScaler().fit_transform(temp_data)
# Clustering:
model = mixture.GaussianMixture(n_components=2, covariance_type='full')  # cluster into 2 categories
clusters = model.fit(scaled_points)

# pcd = o3d.io.read_point_cloud('./bun_zipper.ply')
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(test_data[index, :, :])
# Get points and transform it to a numpy array:
points = np.asarray(pcd.points).copy()

# Get labels:
labels = model.predict(scaled_points)
all_labels = labels
# Get the number of colors:
n_clusters = len(set(labels))
n_noise_ = list(labels).count(-1)
print("Estimated number of clusters: %d" % n_clusters)
print(set(labels))

print(all_labels)

# choose the cluster of reference surface
num_in_clusters = np.array([len(np.where(all_labels == 0)[0]), len(np.where(all_labels == 1)[0])])
print(num_in_clusters)
nums = num_in_clusters[1000 <num_in_clusters]  # according to the S plot, the number of nodes of the reference surface is always larger than 1000
print(nums)
print('The index of the cluster w.r.t. the anomaly surface is:', np.where(num_in_clusters == nums)[0])
new_labels = all_labels .reshape(2048, 1)
index_surface_cluster = np.where(new_labels == np.where(num_in_clusters == nums)[0])[0]
# print(index_surface_cluster)
# print(len(index_surface_cluster))

x0, y0, z0 = test_data[index, index_surface_cluster, 0], test_data[index, index_surface_cluster, 1], test_data[index, index_surface_cluster, 2]
# Solve using SVD method.
a, b, c, d = SVD(np.array([x0, y0, z0]).T)
if a < 0:
  a = -a
  b = -b
  c = -c
  d= -d
print('Current index of sample:', index)
print("SVD  abcd: {:.3f} {:.3f} {:.3f} {:.3f}".format(a, b, c, d))
# Orthogonal mean distance
print("Perp err: {:.5f}\n".format(perp_error((a, b, c, d), (x0, y0, z0))))

# divide nodes according to the plane
coefs = np.array([a, b, c])
dis_vec = (np.dot(test_data[index, :, :], coefs) + d)/(np.sqrt(coefs.dot(coefs)))
print(np.min(dis_vec), np.max(dis_vec))
fig, ax = plt.subplots(figsize =(10, 7))
ax.hist(dis_vec)
threshold = 0.1
index_anomaly_new = np.where(dis_vec > threshold)[0]
index_surface_new = np.where(dis_vec <= threshold)[0]
# print(index_anomaly_new)
# print(index_surface_new)

print('The plot of fitted plane and nodes for the ', index, 'th sample')
x = np.linspace(-0.6, 0.6, 100)
y = np.linspace(-0.6, 0.6, 100)
x, y = np.meshgrid(x, y)
eq = -(a * x + b * y + d)/c
fig = plt.figure()
ax = fig.add_subplot(121, projection='3d')
ax.plot_surface(x, y, eq)
ax.scatter(test_data[index, :, 0], test_data[index, :, 1], test_data[index, :, 2], c='red')
ax.set_xlabel('X', fontsize=16)
ax.set_ylabel('Y', fontsize=16)
ax.set_zlabel('Z', fontsize=16)
ax.view_init(10, 25)
ax.set_title('View 1', y=0.0001, fontsize=15)

ax = fig.add_subplot(122, projection='3d')
ax.plot_surface(x, y, eq)
ax.scatter(test_data[index, :, 0], test_data[index, :, 1], test_data[index, :, 2], c='red')
ax.set_xlabel('X', fontsize=16)
ax.set_ylabel('Y', fontsize=16)
ax.set_zlabel('Z', fontsize=16)
ax.view_init(-10, -25)
ax.set_title('View 2', y=0.0001, fontsize=15)

fig.set_size_inches(13, 6)
fig.suptitle('The fitted reference surface of the sample #{}'.format(index+1), y=0.0001, fontsize=15)
plt.show()


# Mapping the labels classes to a color map:
colors = plt.get_cmap("tab20")(all_labels / (n_clusters if n_clusters > 0 else 1))
# Update points colors:
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_plotly([pcd])

**Segment all samples**:(run the following codes to apply the avove segmentation to all samples)

In [ ]:
import numpy as np
import os
# !pip install open3d==0.16
import open3d as o3d
from sklearn import mixture
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from collections import Counter
import scipy.optimize as optimize
import matplotlib.pyplot as plt


def perp_error(params, xyz):
    a, b, c, d = params
    x, y, z = xyz
    length = np.sqrt(a**2 + b**2 + c**2)
    return (np.abs(a * x + b * y + c * z + d) / length).mean()


def SVD(X):
    # Find the average of points (centroid) along the columns
    C = np.average(X, axis=0)
    # Create CX vector (centroid to point) matrix
    CX = X - C
    # Singular value decomposition
    U, S, V = np.linalg.svd(CX)
    # The last row of V matrix indicate the eigenvectors of
    # smallest eigenvalues (singular values).
    N = V[-1]
    # Extract a, b, c, d coefficients.
    x0, y0, z0 = C
    a, b, c = N
    d = -(a * x0 + b * y0 + c * z0)
    return a, b, c, d

# os.chdir('/content/drive/MyDrive/DAISproject/Datasets')
# # Read point cloud:
# cwd = os.getcwd()
# test_data_path = os.path.join(cwd, 'train_data.npy')
# label_data_path = os.path.join(cwd, 'train_label.npy')

# test_data = np.load(test_data_path)
# label_data = np.load(label_data_path)

data_seg = []
data_seg_surface = []
for index in range(np.shape(test_data)[0]):
    temp_data = local_features_all[index][:, :]

    # Normalisation:
    scaled_points = StandardScaler().fit_transform(temp_data)
    # Clustering:
    model = mixture.GaussianMixture(n_components=2, covariance_type='full')  # cluster into 2 categories
    clusters = model.fit(scaled_points)

    # pcd = o3d.io.read_point_cloud('./bun_zipper.ply')
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(test_data[index, :, :])
    # Get points and transform it to a numpy array:
    points = np.asarray(pcd.points).copy()

    # Get labels:
    labels = model.predict(scaled_points)
    all_labels = labels
    # Get the number of colors:
    n_clusters = len(set(labels))
    n_noise_ = list(labels).count(-1)
    print("Estimated number of clusters: %d" % n_clusters)
    print(set(labels))

    print(all_labels)

    # choose the cluster of reference surface
    num_in_clusters = np.array([len(np.where(all_labels == 0)[0]), len(np.where(all_labels == 1)[0])])
    print(num_in_clusters)
    nums = num_in_clusters[
        1000 < num_in_clusters]  # according to the S plot, the number of nodes of the reference surface is always larger than 1000
    print(nums)
    print('The index of the cluster w.r.t. the anomaly surface is:', np.where(num_in_clusters == nums)[0])
    new_labels = all_labels.reshape(2048, 1)
    index_surface_cluster = np.where(new_labels == np.where(num_in_clusters == nums)[0])[0]
    # print(index_surface_cluster)
    # print(len(index_surface_cluster))

    x0, y0, z0 = test_data[index, index_surface_cluster, 0], test_data[index, index_surface_cluster, 1], test_data[
        index, index_surface_cluster, 2]
    # Solve using SVD method.
    a, b, c, d = SVD(np.array([x0, y0, z0]).T)
    if a < 0:
        a = -a
        b = -b
        c = -c
        d = -d
    # print('Current index of sample:', index)
    # print("SVD  abcd: {:.3f} {:.3f} {:.3f} {:.3f}".format(a, b, c, d))
    # # Orthogonal mean distance
    # print("Perp err: {:.5f}\n".format(perp_error((a, b, c, d), (x0, y0, z0))))

    # divide nodes according to the plane
    coefs = np.array([a, b, c])
    dis_vec = (np.dot(test_data[index, :, :], coefs) + d) / (np.sqrt(coefs.dot(coefs)))
    print(np.min(dis_vec), np.max(dis_vec))
    # fig, ax = plt.subplots(figsize=(10, 7))
    # ax.hist(dis_vec)
    threshold = 0.1
    index_anomaly_new = np.where(dis_vec > threshold)[0]
    index_surface_new = np.where(dis_vec <= threshold)[0]
    data_seg.append(test_data[index, index_anomaly_new, :])
    data_seg_surface.append(test_data[index, index_surface_new, :])

    # Mapping the labels classes to a color map:
    colors = plt.get_cmap("tab20")(all_labels / (n_clusters if n_clusters > 0 else 1))
    # Update points colors:
    pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
    o3d.visualization.draw_plotly([pcd])

    print('The plot of fitted plane and nodes for the ', index, 'th sample')
    x = np.linspace(-0.6, 0.6, 100)
    y = np.linspace(-0.6, 0.6, 100)
    x, y = np.meshgrid(x, y)
    eq = -(a * x + b * y + d) / c
    fig = plt.figure()
    ax = fig.add_subplot(121, projection='3d')
    ax.plot_surface(x, y, eq)
    ax.scatter(data_seg[index][:, 0], data_seg[index][:, 1], data_seg[index][:, 2], c='green')
    ax.scatter(data_seg_surface[index][:, 0], data_seg_surface[index][:, 1], data_seg_surface[index][:, 2], c='red')
    ax.set_xlabel('X', fontsize=16)
    ax.set_ylabel('Y', fontsize=16)
    ax.set_zlabel('Z', fontsize=16)
    ax.view_init(0, -140)
    ax.set_title('View 1', y=0.0001, fontsize=15)

    ax = fig.add_subplot(122, projection='3d')
    ax.plot_surface(x, y, eq)
    ax.scatter(data_seg[index][:, 0], data_seg[index][:, 1], data_seg[index][:, 2], c='green')
    ax.scatter(data_seg_surface[index][:, 0], data_seg_surface[index][:, 1], data_seg_surface[index][:, 2], c='red')
    ax.set_xlabel('X', fontsize=16)
    ax.set_ylabel('Y', fontsize=16)
    ax.set_zlabel('Z', fontsize=16)
    ax.view_init(-10, 25)
    ax.set_title('View 2', y=0.0001, fontsize=15)

    fig.set_size_inches(13, 6)
    fig.suptitle('The fitted reference surface of the sample #{}'.format(index + 1), y=0.0001, fontsize=15)
    plt.show()

# the list 'data_seg' contains the anomaly surface nodes for each samples, but it is noted that the numbers of nodes are not consistent

For test dataset: **Segment all samples**:(run the following codes to apply the avove segmentation to all samples)

In [ ]:
import numpy as np
import os
# !pip install open3d==0.16
import open3d as o3d
from sklearn import mixture
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from collections import Counter
import scipy.optimize as optimize
import matplotlib.pyplot as plt


def perp_error(params, xyz):
    a, b, c, d = params
    x, y, z = xyz
    length = np.sqrt(a**2 + b**2 + c**2)
    return (np.abs(a * x + b * y + c * z + d) / length).mean()


def SVD(X):
    # Find the average of points (centroid) along the columns
    C = np.average(X, axis=0)
    # Create CX vector (centroid to point) matrix
    CX = X - C
    # Singular value decomposition
    U, S, V = np.linalg.svd(CX)
    # The last row of V matrix indicate the eigenvectors of
    # smallest eigenvalues (singular values).
    N = V[-1]
    # Extract a, b, c, d coefficients.
    x0, y0, z0 = C
    a, b, c = N
    d = -(a * x0 + b * y0 + c * z0)
    return a, b, c, d

data_seg_Test = []
data_seg_surface_Test = []
for index in range(np.shape(test_data_Test)[0]):
    temp_data = local_features_all_Test[index][:, :]

    # Normalisation:
    scaled_points = StandardScaler().fit_transform(temp_data)
    # Clustering:
    model = mixture.GaussianMixture(n_components=2, covariance_type='full')  # cluster into 2 categories
    clusters = model.fit(scaled_points)

    # pcd = o3d.io.read_point_cloud('./bun_zipper.ply')
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(test_data_Test[index, :, :])
    # Get points and transform it to a numpy array:
    points = np.asarray(pcd.points).copy()

    # Get labels:
    labels = model.predict(scaled_points)
    all_labels = labels
    # Get the number of colors:
    n_clusters = len(set(labels))
    n_noise_ = list(labels).count(-1)
    print("Estimated number of clusters: %d" % n_clusters)
    print(set(labels))

    print(all_labels)

    # choose the cluster of reference surface
    num_in_clusters = np.array([len(np.where(all_labels == 0)[0]), len(np.where(all_labels == 1)[0])])
    print(num_in_clusters)
    nums = num_in_clusters[
        1000 < num_in_clusters]  # according to the S plot, the number of nodes of the reference surface is always larger than 1000
    print(nums)
    print('The index of the cluster w.r.t. the anomaly surface is:', np.where(num_in_clusters == nums)[0])
    new_labels = all_labels.reshape(2048, 1)
    index_surface_cluster = np.where(new_labels == np.where(num_in_clusters == nums)[0])[0]
    # print(index_surface_cluster)
    # print(len(index_surface_cluster))

    x0, y0, z0 = test_data_Test[index, index_surface_cluster, 0], test_data_Test[index, index_surface_cluster, 1], test_data_Test[
        index, index_surface_cluster, 2]
    # Solve using SVD method.
    a, b, c, d = SVD(np.array([x0, y0, z0]).T)
    if a < 0:
        a = -a
        b = -b
        c = -c
        d = -d
    # print('Current index of sample:', index)
    # print("SVD  abcd: {:.3f} {:.3f} {:.3f} {:.3f}".format(a, b, c, d))
    # # Orthogonal mean distance
    # print("Perp err: {:.5f}\n".format(perp_error((a, b, c, d), (x0, y0, z0))))

    # divide nodes according to the plane
    coefs = np.array([a, b, c])
    dis_vec = (np.dot(test_data_Test[index, :, :], coefs) + d) / (np.sqrt(coefs.dot(coefs)))
    print(np.min(dis_vec), np.max(dis_vec))
    # fig, ax = plt.subplots(figsize=(10, 7))
    # ax.hist(dis_vec)
    threshold = 0.1
    index_anomaly_new = np.where(dis_vec > threshold)[0]
    index_surface_new = np.where(dis_vec <= threshold)[0]
    data_seg_Test.append(test_data_Test[index, index_anomaly_new, :])
    data_seg_surface_Test.append(test_data_Test[index, index_surface_new, :])

    # Mapping the labels classes to a color map:
    colors = plt.get_cmap("tab20")(all_labels / (n_clusters if n_clusters > 0 else 1))
    # Update points colors:
    pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
    o3d.visualization.draw_plotly([pcd])

    print('The plot of fitted plane and nodes for the ', index, 'th sample')
    x = np.linspace(-0.6, 0.6, 100)
    y = np.linspace(-0.6, 0.6, 100)
    x, y = np.meshgrid(x, y)
    eq = -(a * x + b * y + d) / c
    fig = plt.figure()
    ax = fig.add_subplot(121, projection='3d')
    ax.plot_surface(x, y, eq)
    ax.scatter(data_seg_Test[index][:, 0], data_seg_Test[index][:, 1], data_seg_Test[index][:, 2], c='green')
    ax.scatter(data_seg_surface_Test[index][:, 0], data_seg_surface_Test[index][:, 1], data_seg_surface_Test[index][:, 2], c='red')
    ax.set_xlabel('X', fontsize=16)
    ax.set_ylabel('Y', fontsize=16)
    ax.set_zlabel('Z', fontsize=16)
    ax.view_init(0, -140)
    ax.set_title('View 1', y=0.0001, fontsize=15)

    ax = fig.add_subplot(122, projection='3d')
    ax.plot_surface(x, y, eq)
    ax.scatter(data_seg_Test[index][:, 0], data_seg_Test[index][:, 1], data_seg_Test[index][:, 2], c='green')
    ax.scatter(data_seg_surface_Test[index][:, 0], data_seg_surface_Test[index][:, 1], data_seg_surface_Test[index][:, 2], c='red')
    ax.set_xlabel('X', fontsize=16)
    ax.set_ylabel('Y', fontsize=16)
    ax.set_zlabel('Z', fontsize=16)
    ax.view_init(-10, 25)
    ax.set_title('View 2', y=0.0001, fontsize=15)

    fig.set_size_inches(13, 6)
    fig.suptitle('The fitted reference surface of the sample #{}'.format(index + 1), y=0.0001, fontsize=15)
    plt.show()

# the list 'data_seg' contains the anomaly surface nodes for each samples, but it is noted that the numbers of nodes are not consistent

Aligh all samples by increasing the number of points: (for Wasserstein Distance and the PointNet model) 

*   step 1. randomly choose one existing node;
*   step 2. find K-nearest neighborhoods;
*   step 3. use them to intepolate a new node;
*   step 4. repeat 1-3 many times as needed.






     

In [ ]:
import numpy as np
import random
# !pip install open3d
import open3d as o3d
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import proj3d
from mpl_toolkits.mplot3d import Axes3D

def augmentPoints(dataset_temp, num_new, radius):
  pcd = o3d.geometry.PointCloud()
  # print(np.shape(dataset_temp))
  # dataset_temp = dataset_temp[0, :, :]
  new_dataset = dataset_temp
  pcd.points = o3d.utility.Vector3dVector(dataset_temp)
  kdtree = o3d.geometry.KDTreeFlann(pcd)
  for i in range(num_new):
    index_temp = random.randint(0, np.shape(dataset_temp)[0]-1)
    radius_temp = radius
    k = 0
    while k < 3:
      radius_temp = radius_temp*1.1
      k, idx, _ = kdtree.search_radius_vector_3d(pcd.points[index_temp], radius_temp)
    neighbors = dataset_temp[idx, :]
    mean = np.mean(neighbors, 0)
    new_dataset = np.vstack((new_dataset, mean.reshape((1, 3))))
    # print(k)
  # print(np.shape(new_dataset))
  return new_dataset


# first, find the maximum number of nodes among all samples
num_nodes_all = []
for i in range(np.shape(test_data)[0]): num_nodes_all.append(np.shape(np.array(data_seg[i]))[0])
# print(num_nodes_all)
max_num = max(num_nodes_all)
print('The maximum number of nodes in different samples:', max_num)
# initial search radius, which is adaptively increased later
radius_para = 0.00001
new_dataset_all = np.zeros((190, max_num, 3))
for j in range(np.shape(test_data)[0]): new_dataset_all[j, :, :]= augmentPoints(np.array(data_seg[j]), max_num-num_nodes_all[j], radius_para)
# print(np.shape(new_dataset_all))  # print the shape of anomaly surface nodes

# here we save the anomaly surface data from segmentation
np.save('new_train_data.npy', new_dataset_all)

Allign the testing dataset

In [ ]:
import numpy as np
import random
# !pip install open3d
import open3d as o3d
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import proj3d
from mpl_toolkits.mplot3d import Axes3D

def augmentPoints(dataset_temp, num_new, radius):
  pcd = o3d.geometry.PointCloud()
  # print(np.shape(dataset_temp))
  # dataset_temp = dataset_temp[0, :, :]
  new_dataset = dataset_temp
  pcd.points = o3d.utility.Vector3dVector(dataset_temp)
  kdtree = o3d.geometry.KDTreeFlann(pcd)
  for i in range(num_new):
    index_temp = random.randint(0, np.shape(dataset_temp)[0]-1)
    radius_temp = radius
    k = 0
    while k < 3:
      radius_temp = radius_temp*1.1
      k, idx, _ = kdtree.search_radius_vector_3d(pcd.points[index_temp], radius_temp)
    neighbors = dataset_temp[idx, :]
    mean = np.mean(neighbors, 0)
    new_dataset = np.vstack((new_dataset, mean.reshape((1, 3))))
    # print(k)
  # print(np.shape(new_dataset))
  return new_dataset


# first, find the maximum number of nodes among all samples
num_nodes_all_Test = []
for i in range(np.shape(test_data_Test)[0]): num_nodes_all_Test.append(np.shape(np.array(data_seg_Test[i]))[0])
# print(num_nodes_all)
max_num = max(num_nodes_all)
print('The maximum number of nodes in different samples:', max_num)
# initial search radius, which is adaptively increased later
radius_para = 0.00001
new_dataset_all_Test = np.zeros((np.shape(test_data_Test)[0], max_num, 3))
for j in range(np.shape(test_data_Test)[0]): new_dataset_all_Test[j, :, :]= augmentPoints(np.array(data_seg_Test[j]), max_num-num_nodes_all_Test[j], radius_para)
# print(np.shape(new_dataset_all))  # print the shape of anomaly surface nodes

# here we save the anomaly surface data from segmentation
np.save('new_train_data_Test.npy', new_dataset_all_Test)

Plot the anomaly surface example: (for **test dataset**)

In [ ]:
print(np.shape(new_dataset_all_Test))

all_index = np.shape(new_dataset_all_Test)[0]

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
for id in range(all_index):
  x0 = new_dataset_all_Test[id,:,0]
  y0 = new_dataset_all_Test[id,:,1]
  z0 = new_dataset_all_Test[id,:,2]
  ax.scatter(x0, y0, z0, c='red')
# plt.show()

fig = plt.figure(figsize=(8, 8))
axes3d = fig.add_subplot(111, projection='3d')
for id in range(all_index):
  x0 = data_seg_Test[id][:,0]
  y0 = data_seg_Test[id][:,1]
  z0 = data_seg_Test[id][:,2]
  axes3d.scatter(x0, y0, z0, c='red')
plt.show()

# **Feature Engineering**

**Feature 1:** (the vertical distance to the reference surface) N = 7

In [ ]:
# Fit the reference surface (i.e., the plane), using list 'data_seg_surface'
import numpy as np
import scipy.optimize as optimize
import matplotlib.pyplot as plt
import os
from mpl_toolkits.mplot3d import proj3d
from mpl_toolkits.mplot3d import Axes3D

# os.chdir('/content/drive/MyDrive/DAISproject/Datasets')
# cwd = os.getcwd()
# # print(cwd)
# test_data_path = os.path.join(cwd, 'train_data.npy')
# label_data_path = os.path.join(cwd, 'train_label.npy')

# test_data = np.load(test_data_path)
# label_data = np.load(label_data_path)


def perp_error(params, xyz):
    """
    Mean of the absolute values for the perpendicular distance of the
    'xyz' points, to the plane defined by the coefficients 'a,b,c,d' in
    'params'.
    """
    a, b, c, d = params
    x, y, z = xyz
    length = np.sqrt(a**2 + b**2 + c**2)
    return (np.abs(a * x + b * y + c * z + d) / length).mean()


def SVD(X):
    """
    Singular value decomposition method.
    Source: https://gist.github.com/lambdalisue/7201028
    """
    # Find the average of points (centroid) along the columns
    C = np.average(X, axis=0)

    # Create CX vector (centroid to point) matrix
    CX = X - C
    # Singular value decomposition
    U, S, V = np.linalg.svd(CX)
    # The last row of V matrix indicate the eigenvectors of
    # smallest eigenvalues (singular values).
    N = V[-1]

    # Extract a, b, c, d coefficients.
    x0, y0, z0 = C
    a, b, c = N
    d = -(a * x0 + b * y0 + c * z0)

    return a, b, c, d

# calculate the vertical distances
vertical_distance_all = []

# mean error of surface fitting
error_fit = []

for index in range(np.shape(test_data)[0]):
  x0, y0, z0 = data_seg_surface[index][:, 0], data_seg_surface[index][:, 1], data_seg_surface[index][:, 2]
  # Solve using SVD method.
  a, b, c, d = SVD(np.array([x0, y0, z0]).T)
  # print('Current index of sample:', index)
  # print("SVD  abcd: {:.3f} {:.3f} {:.3f} {:.3f}".format(a, b, c, d))
  # # Orthogonal mean distance
  # print("Perp err: {:.5f}\n".format(perp_error((a, b, c, d), (x0, y0, z0))))
  error_fit.append(perp_error((a, b, c, d), (x0, y0, z0)))
  # calculate the vertical distance
  n_d, _ = np.shape(data_seg[index])
  vertical_distance = -(a * data_seg[index][:, 0] + b * data_seg[index][:, 1] + d)/c - data_seg[index][:, 2]
  vertical_distance_all.append(vertical_distance)

print('The plot of fitted plane and nodes for the 190th sample:')
# plot the plane with nodes for the 190th sample
plt.rcParams["figure.figsize"] = [14.00, 3.50]
plt.rcParams["figure.autolayout"] = True
x = np.linspace(-0.6, 0.6, 100)
y = np.linspace(-0.6, 0.6, 100)
x, y = np.meshgrid(x, y)
eq = -(a * x + b * y + d)/c
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(x, y, eq)
ax.scatter(x0, y0, z0, c='red')
ax.view_init(30, 45)
plt.show()

**For test dataset:**

In [ ]:
# Fit the reference surface (i.e., the plane), using list 'data_seg_surface'
import numpy as np
import scipy.optimize as optimize
import matplotlib.pyplot as plt
import os
from mpl_toolkits.mplot3d import proj3d
from mpl_toolkits.mplot3d import Axes3D


def perp_error(params, xyz):
    """
    Mean of the absolute values for the perpendicular distance of the
    'xyz' points, to the plane defined by the coefficients 'a,b,c,d' in
    'params'.
    """
    a, b, c, d = params
    x, y, z = xyz
    length = np.sqrt(a**2 + b**2 + c**2)
    return (np.abs(a * x + b * y + c * z + d) / length).mean()


def SVD(X):
    """
    Singular value decomposition method.
    Source: https://gist.github.com/lambdalisue/7201028
    """
    # Find the average of points (centroid) along the columns
    C = np.average(X, axis=0)

    # Create CX vector (centroid to point) matrix
    CX = X - C
    # Singular value decomposition
    U, S, V = np.linalg.svd(CX)
    # The last row of V matrix indicate the eigenvectors of
    # smallest eigenvalues (singular values).
    N = V[-1]

    # Extract a, b, c, d coefficients.
    x0, y0, z0 = C
    a, b, c = N
    d = -(a * x0 + b * y0 + c * z0)

    return a, b, c, d

# calculate the vertical distances
vertical_distance_all_Test = []

# mean error of surface fitting
error_fit_Test = []

for index in range(np.shape(test_data_Test)[0]):
  x0, y0, z0 = data_seg_surface_Test[index][:, 0], data_seg_surface_Test[index][:, 1], data_seg_surface_Test[index][:, 2]
  # Solve using SVD method.
  a, b, c, d = SVD(np.array([x0, y0, z0]).T)
  # print('Current index of sample:', index)
  # print("SVD  abcd: {:.3f} {:.3f} {:.3f} {:.3f}".format(a, b, c, d))
  # # Orthogonal mean distance
  # print("Perp err: {:.5f}\n".format(perp_error((a, b, c, d), (x0, y0, z0))))
  error_fit_Test.append(perp_error((a, b, c, d), (x0, y0, z0)))
  # calculate the vertical distance
  n_d, _ = np.shape(data_seg_Test[index])
  vertical_distance = -(a * data_seg_Test[index][:, 0] + b * data_seg_Test[index][:, 1] + d)/c - data_seg_Test[index][:, 2]
  vertical_distance_all_Test.append(vertical_distance)

print('The plot of fitted plane and nodes for the last testing sample:')
# plot the plane with nodes for the 190th sample
plt.rcParams["figure.figsize"] = [14.00, 3.50]
plt.rcParams["figure.autolayout"] = True
x = np.linspace(-0.6, 0.6, 100)
y = np.linspace(-0.6, 0.6, 100)
x, y = np.meshgrid(x, y)
eq = -(a * x + b * y + d)/c
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(x, y, eq)
ax.scatter(x0, y0, z0, c='red')
ax.view_init(30, 45)
plt.show()

calculate the mean and the quantiles of the distribution of the vertical distance: (quantiles are 0.025, 0.25, 0.50, 0.75, 0.975, 1)

In [ ]:
# Feature 1
# the mean of the vertical distance
mean_vertiDistance_all = []
# the quantiles of the distribution of the vertical distance
quantiles_vertiDistance_all = []
for i in range(np.shape(test_data)[0]): mean_vertiDistance_all.append(np.mean(vertical_distance_all[i])/np.shape(vertical_distance_all[i]))
for i in range(np.shape(test_data)[0]): quantiles_vertiDistance_all.append([np.quantile(vertical_distance_all[i], 0.025), np.quantile(vertical_distance_all[i], 0.25), np.quantile(vertical_distance_all[i], 0.5), np.quantile(vertical_distance_all[i], 0.75), np.quantile(vertical_distance_all[i], 0.975), np.quantile(vertical_distance_all[i], 1)])
# print(mean_vertiDistance_all)
# print(quantiles_vertiDistance_all)
feature_1_mean = mean_vertiDistance_all
feature_1_quantiles = quantiles_vertiDistance_all

In [ ]:
# Feature 1 for testing dataset
# the mean of the vertical distance
mean_vertiDistance_all_Test = []
# the quantiles of the distribution of the vertical distance
quantiles_vertiDistance_all_Test = []
for i in range(np.shape(test_data_Test)[0]): mean_vertiDistance_all_Test.append(np.mean(vertical_distance_all_Test[i])/np.shape(vertical_distance_all_Test[i]))
for i in range(np.shape(test_data_Test)[0]): quantiles_vertiDistance_all_Test.append([np.quantile(vertical_distance_all_Test[i], 0.025), np.quantile(vertical_distance_all_Test[i], 0.25), np.quantile(vertical_distance_all_Test[i], 0.5), np.quantile(vertical_distance_all_Test[i], 0.75), np.quantile(vertical_distance_all_Test[i], 0.975), np.quantile(vertical_distance_all_Test[i], 1)])
# print(mean_vertiDistance_all)
# print(quantiles_vertiDistance_all)
feature_1_mean_Test = mean_vertiDistance_all_Test
feature_1_quantiles_Test = quantiles_vertiDistance_all_Test

**Feature 2:** (the mean and the deviation of the coordinates of anomaly surface nodes) N = 6

In [ ]:
# Feature 2
# the mean of the coordinates of anomaly surface nodes
feature_2_mean = []
feature_2_deviation = []
for i in range(np.shape(test_data)[0]): feature_2_mean.append([np.mean(data_seg[i][:, 0]), np.mean(data_seg[i][:, 1]), np.mean(data_seg[i][:, 2])])
for i in range(np.shape(test_data)[0]): feature_2_deviation.append([np.std(data_seg[i][:, 0]), np.std(data_seg[i][:, 1]), np.std(data_seg[i][:, 2])])
# print(feature_2_mean)
# print(feature_2_deviation)

In [ ]:
# Feature 2 for testing dataset
# the mean of the coordinates of anomaly surface nodes
feature_2_mean_Test = []
feature_2_deviation_Test = []
for i in range(np.shape(test_data_Test)[0]): feature_2_mean_Test.append([np.mean(data_seg_Test[i][:, 0]), np.mean(data_seg_Test[i][:, 1]), np.mean(data_seg_Test[i][:, 2])])
for i in range(np.shape(test_data_Test)[0]): feature_2_deviation_Test.append([np.std(data_seg_Test[i][:, 0]), np.std(data_seg_Test[i][:, 1]), np.std(data_seg_Test[i][:, 2])])

**Feature 3:** (the size of the dent) N = 2

In [ ]:
# Feature 3
feature_3_length = []
feature_3_width = []
for i in range(np.shape(test_data)[0]): feature_3_length.append(np.max(data_seg[i][:, 0]) - np.min(data_seg[i][:, 0]))
for i in range(np.shape(test_data)[0]): feature_3_width.append(np.max(data_seg[i][:, 1]) - np.min(data_seg[i][:, 1]))
# print(feature_3_length)
# print(feature_3_width)

In [ ]:
# Feature 3
feature_3_length_Test = []
feature_3_width_Test = []
for i in range(np.shape(test_data_Test)[0]): feature_3_length_Test.append(np.max(data_seg_Test[i][:, 0]) - np.min(data_seg_Test[i][:, 0]))
for i in range(np.shape(test_data_Test)[0]): feature_3_width_Test.append(np.max(data_seg_Test[i][:, 1]) - np.min(data_seg_Test[i][:, 1]))

**Feature 4:** (the number of the anomaly surface nodes) N = 1

In [ ]:
# Feature 4
feature_4 = []
for i in range(np.shape(test_data)[0]): feature_4.append(np.shape(data_seg[i])[0])
# print(feature_4)

In [ ]:
# Feature 4
feature_4_Test = []
for i in range(np.shape(test_data_Test)[0]): feature_4_Test.append(np.shape(data_seg_Test[i])[0])

**Feature 5:** (the mean and the deviation of the coordinates of reference surface nodes) N = 6

In [ ]:
# Feature 5
# the mean of the coordinates of anomaly surface nodes
feature_5_mean = []
feature_5_deviation = []
for i in range(np.shape(test_data)[0]): feature_5_mean.append([np.mean(data_seg_surface[i][:, 0]), np.mean(data_seg_surface[i][:, 1]), np.mean(data_seg_surface[i][:, 2])])
for i in range(np.shape(test_data)[0]): feature_5_deviation.append([np.std(data_seg_surface[i][:, 0]), np.std(data_seg_surface[i][:, 1]), np.std(data_seg_surface[i][:, 2])])
# print(feature_5_mean)
# print(feature_5_deviation)

In [ ]:
# Feature 5
# the mean of the coordinates of anomaly surface nodes
feature_5_mean_Test = []
feature_5_deviation_Test = []
for i in range(np.shape(test_data_Test)[0]): feature_5_mean_Test.append([np.mean(data_seg_surface_Test[i][:, 0]), np.mean(data_seg_surface_Test[i][:, 1]), np.mean(data_seg_surface_Test[i][:, 2])])
for i in range(np.shape(test_data_Test)[0]): feature_5_deviation_Test.append([np.std(data_seg_surface_Test[i][:, 0]), np.std(data_seg_surface_Test[i][:, 1]), np.std(data_seg_surface_Test[i][:, 2])])

**Feature 6:** (the mean distance of nodes around surface (to the fitted surface) )

In [ ]:
# Feature 6
feature_6_fitErr = []
for i in range(np.shape(test_data)[0]): feature_6_fitErr.append(error_fit[i])
# print(feature_6_fitErr)

In [ ]:
# Feature 6
feature_6_fitErr_Test = []
for i in range(np.shape(test_data_Test)[0]): feature_6_fitErr_Test.append(error_fit_Test[i])

**Feature 7:** (the Hausdorff Distance)

In [ ]:
!pip install point-cloud-utils

In [ ]:
# Feature 7
import point_cloud_utils as pcu
import numpy as np
import matplotlib.pylab as plt
import scipy.sparse as sparse
import os


index_0 = np.where(label_data == 0)[0]
index_1 = np.where(label_data == 1)[0]

# Hausdorff distance
hausdorff_dist_matrix = np.zeros((190, 190))
for i in list(index_0) + list(index_1):
  for j in list(index_0) + list(index_1):
    dataset_class0 = data_seg[i]
    dataset_class1 = data_seg[j]
    # Take a max of the one sided squared  distances to get the two sided Hausdorff distance
    hausdorff_dist_matrix[i, j] = pcu.hausdorff_distance(dataset_class0, dataset_class1)
# visualize the sparse matrix with Spy
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
ax.matshow(hausdorff_dist_matrix)

# Chamfer distance
chamfer_dist_matrix = chamfer_dist_matrix = np.zeros((190, 190))
for i in list(index_0) + list(index_1):
  for j in list(index_0) + list(index_1):
    dataset_class0 = data_seg[i]
    dataset_class1 = data_seg[j]
    # Take a max of the one sided squared  distances to get the two sided Hausdorff distance
    # chamfer_dist_matrix[i, j] = pcu.hausdorff_distance(dataset_class0, dataset_class1)
    chamfer_dist_matrix[i, j] = pcu.chamfer_distance(dataset_class0, dataset_class1)
# visualize the sparse matrix with Spy
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
ax.matshow(chamfer_dist_matrix)

# # print(np.shape(hausdorff_dist_matrix))

# # # Approximate Wasserstein (Sinkhorn) distance
# # Wasserstein_dist_matrix = np.zeros((190, 190))
# # for i in list(index_0) + list(index_1):
# #   for j in list(index_0) + list(index_1):
# #     dataset_class0 = data_seg[i]
# #     dataset_class1 = data_seg[j]
# #     # a and b are arrays where each row contains a point
# #     # Note that the point sets can have different sizes (e.g [100, 3], [111, 3])
# #     a = dataset_class0
# #     b = dataset_class1

# #     # M is a 100x100 array where each entry  (i, j) is the L2 distance between point a[i, :] and b[j, :]
# #     M = pcu.pairwise_distances(a, b)
# #     M = M.astype('float64')

# #     # w_a and w_b are masses assigned to each point. In this case each point is weighted equally.
# #     w_a = np.ones(a.shape[0])
# #     w_b = np.ones(b.shape[0])

# #     # P is the transport matrix between a and b, eps is a regularization parameter, smaller epsilons lead to
# #     # better approximation of the true Wasserstein distance at the expense of slower convergence
# #     P = pcu.sinkhorn(w_a, w_b, M, eps=1e-3)

# #     # To get the distance as a number just compute the frobenius inner product <M, P>
# #     sinkhorn_dist = (M*P).sum()
# #     Wasserstein_dist_matrix[i, j] = sinkhorn_dist
# # # visualize the sparse matrix with Spy
# # fig = plt.figure(figsize=(10, 10))
# # ax = fig.add_subplot(111)
# # ax.matshow(Wasserstein_dist_matrix)

# # np.save('Wasserstein_distance.npy', hausdorff_dist_matrix)

**For test dataset:**

In [ ]:
# Feature 7 for test dataset
import point_cloud_utils as pcu
import numpy as np
import matplotlib.pylab as plt
import scipy.sparse as sparse
import os


index_0 = np.where(label_data == 0)[0]
index_1 = np.where(label_data == 1)[0]

num_Test = np.shape(test_data_Test)[0]

# Hausdorff distance
hausdorff_dist_matrix_Test = np.zeros((190, num_Test))
for i in list(index_0) + list(index_1):
  for j in range(num_Test):
    dataset_class0 = data_seg[i]
    dataset_class1 = data_seg_Test[j]
    # Take a max of the one sided squared  distances to get the two sided Hausdorff distance
    hausdorff_dist_matrix_Test[i, j] = pcu.hausdorff_distance(dataset_class0, dataset_class1)
# visualize the sparse matrix with Spy
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
ax.matshow(hausdorff_dist_matrix_Test)

# Chamfer distance
chamfer_dist_matrix_Test = np.zeros((190, num_Test))
for i in list(index_0) + list(index_1):
  for j in range(num_Test):
    dataset_class0 = data_seg[i]
    dataset_class1 = data_seg_Test[j]
    # Take a max of the one sided squared  distances to get the two sided Hausdorff distance
    # chamfer_dist_matrix[i, j] = pcu.hausdorff_distance(dataset_class0, dataset_class1)
    chamfer_dist_matrix_Test[i, j] = pcu.chamfer_distance(dataset_class0, dataset_class1)
# visualize the sparse matrix with Spy
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
ax.matshow(chamfer_dist_matrix_Test)

Using augmented data

In [ ]:
# import os
# import numpy as np
# BASE_DIR = os.path.dirname("C:/Users/xl037/PycharmProjects/DAISproject/")
# DATA_DIR = os.path.join(BASE_DIR, 'Datasets')
# train_data_path = os.path.join(DATA_DIR, 'new_train_data.npy')
# new_data = np.load(train_data_path)

# # Approximate Wasserstein (Sinkhorn) distance
# Wasserstein_dist_matrix = np.zeros((190, 190))
# num_ = 0
# for i in list(index_0) + list(index_1):
#   print(num_)
#   num_ += 1
#   for j in list(index_0) + list(index_1):
#     dataset_class0 = new_data[i, :, :]
#     dataset_class1 = new_data[j, :, :]
#     # a and b are arrays where each row contains a point
#     # Note that the point sets can have different sizes (e.g [100, 3], [111, 3])
#     a = dataset_class0
#     b = dataset_class1

#     # M is a 100x100 array where each entry  (i, j) is the L2 distance between point a[i, :] and b[j, :]
#     M = pcu.pairwise_distances(a, b)
#     M = M.astype('float64')

#     # w_a and w_b are masses assigned to each point. In this case each point is weighted equally.
#     w_a = np.ones(a.shape[0])
#     w_b = np.ones(b.shape[0])

#     # P is the transport matrix between a and b, eps is a regularization parameter, smaller epsilons lead to
#     # better approximation of the true Wasserstein distance at the expense of slower convergence
#     P = pcu.sinkhorn(w_a, w_b, M, eps=1e-3)

#     # To get the distance as a number just compute the frobenius inner product <M, P>
#     sinkhorn_dist = (M*P).sum()
#     Wasserstein_dist_matrix[i, j] = sinkhorn_dist
# # visualize the sparse matrix with Spy
# fig = plt.figure(figsize=(10, 10))
# ax = fig.add_subplot(111)
# ax.matshow(Wasserstein_dist_matrix)
# np.save('aug_Wasserstein_distance.npy', Wasserstein_dist_matrix)

**for test dataset:**

In [ ]:
import os
import numpy as np

cwd = os.getcwd()
train_data_path = os.path.join(cwd, 'new_train_data.npy')
train_data_Test_path = os.path.join(cwd, 'new_train_data_Test.npy')
new_data = np.load(train_data_path)
new_data_Test = np.load(train_data_Test_path)

num_Test = np.shape(test_data_Test)[0]

# Approximate Wasserstein (Sinkhorn) distance
Wasserstein_dist_matrix_Test = np.zeros((190, num_Test))
num_ = 0
for i in list(index_0) + list(index_1):
  print(num_)
  num_ += 1
  for j in range(num_Test):
    dataset_class0 = new_data[i, :, :]
    dataset_class1 = new_data_Test[j, :, :]
    # a and b are arrays where each row contains a point
    # Note that the point sets can have different sizes (e.g [100, 3], [111, 3])
    a = dataset_class0
    b = dataset_class1

    # M is a 100x100 array where each entry  (i, j) is the L2 distance between point a[i, :] and b[j, :]
    M = pcu.pairwise_distances(a, b)
    M = M.astype('float64')

    # w_a and w_b are masses assigned to each point. In this case each point is weighted equally.
    w_a = np.ones(a.shape[0])
    w_b = np.ones(b.shape[0])

    # P is the transport matrix between a and b, eps is a regularization parameter, smaller epsilons lead to
    # better approximation of the true Wasserstein distance at the expense of slower convergence
    P = pcu.sinkhorn(w_a, w_b, M, eps=1e-3)

    # To get the distance as a number just compute the frobenius inner product <M, P>
    sinkhorn_dist = (M*P).sum()
    Wasserstein_dist_matrix_Test[i, j] = sinkhorn_dist
# visualize the sparse matrix with Spy
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
ax.matshow(Wasserstein_dist_matrix_Test)

In [ ]:
np.save('aug_Wasserstein_distance_Test.npy', Wasserstein_dist_matrix_Test)

In [ ]:
# os.chdir('/content/drive/MyDrive/DAISproject/Datasets')
# print(cwd)
test_data_path = os.path.join(cwd, 'aug_Wasserstein_distance.npy')
Wasserstein_dist_matrix = np.load(test_data_path)
# visualize the sparse matrix with Spy
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
ax.matshow(Wasserstein_dist_matrix)

print(np.shape(Wasserstein_dist_matrix))

**Combine all the features:**

In [ ]:
from sklearn import preprocessing


# define the array for saving all features
num_features = 23 + 4*3
feature_all = np.zeros((190, num_features))
Num_samples = 190
# feature 1
for i in range(Num_samples): feature_all[i, 0:1] = feature_1_mean[i]
for i in range(Num_samples): feature_all[i, 1:7] = feature_1_quantiles[i]
# feature 2
for i in range(Num_samples): feature_all[i, 7:10] = feature_2_mean[i]
for i in range(Num_samples): feature_all[i, 10:13] = feature_2_deviation[i]
# feature 3
for i in range(Num_samples): feature_all[i, 13:14] = feature_3_length[i]
for i in range(Num_samples): feature_all[i, 14:15] = feature_3_width[i]
# feature 4
for i in range(Num_samples): feature_all[i, 15:16] = feature_4[i]
# feature 5
for i in range(Num_samples): feature_all[i, 16:19] = feature_5_mean[i]
for i in range(Num_samples): feature_all[i, 19:22] = feature_5_deviation[i]
#feature 6
for i in range(Num_samples): feature_all[i, 22:23] = feature_6_fitErr[i]
# print(feature_all)
print(np.shape(test_data)[0])
# normalize all the data
for i in range(num_features):
  feature_all[:, i] = preprocessing.normalize([feature_all[:, i]])
# print(feature_all)
# print(feature_all[:, 36:37])

**for test dataset:**

In [ ]:
from sklearn import preprocessing


# define the array for saving all features
num_features = 23 + 4*3
feature_all_Test = np.zeros((np.shape(test_data_Test)[0], num_features))
Num_samples = np.shape(test_data_Test)[0]
# feature 1
for i in range(Num_samples): feature_all_Test[i, 0:1] = feature_1_mean_Test[i]
for i in range(Num_samples): feature_all_Test[i, 1:7] = feature_1_quantiles_Test[i]
# feature 2
for i in range(Num_samples): feature_all_Test[i, 7:10] = feature_2_mean_Test[i]
for i in range(Num_samples): feature_all_Test[i, 10:13] = feature_2_deviation_Test[i]
# feature 3
for i in range(Num_samples): feature_all_Test[i, 13:14] = feature_3_length_Test[i]
for i in range(Num_samples): feature_all_Test[i, 14:15] = feature_3_width_Test[i]
# feature 4
for i in range(Num_samples): feature_all_Test[i, 15:16] = feature_4_Test[i]
# feature 5
for i in range(Num_samples): feature_all_Test[i, 16:19] = feature_5_mean_Test[i]
for i in range(Num_samples): feature_all_Test[i, 19:22] = feature_5_deviation_Test[i]
#feature 6
for i in range(Num_samples): feature_all_Test[i, 22:23] = feature_6_fitErr_Test[i]
# print(feature_all)
print(np.shape(test_data_Test)[0])
# normalize all the data
for i in range(num_features):
  feature_all_Test[:, i] = preprocessing.normalize([feature_all_Test[:, i]])

In [ ]:
# correlation without advanced distance metrics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def correlation_heatmap(train):
    correlations = np.corrcoef(train)

    fig, ax = plt.subplots(figsize=(50,50))
    sns.heatmap(correlations, vmax=1.0, center=0, fmt='.2f',
                square=True, linewidths=.5, annot=True, annot_kws={"fontsize":21}, cbar_kws={"shrink": .70})
    plt.show()

train = np.vstack((feature_all[:, 0:23].reshape(23, 190), label_data.reshape(1, 190)))
# train = np.vstack((feature_all[:, 35:36].reshape(1, 190), label_data.reshape(1, 190)))
correlation_heatmap(train)

# **Balancing, Training, and Classification Prediction (XGboost):**

In [ ]:
!pip install imbalanced-learn
!pip install xgboost

In [ ]:
# repeat the training and prediction process many times
# use XGboost
# without flip the 0 and 1
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
import point_cloud_utils as pcu
import numpy as np
import matplotlib.pylab as plt
import scipy.sparse as sparse
import random
import statistics
from imblearn.over_sampling import SMOTE


all_data = np.load(test_data_Train_path)
all_label = np.load(label_data_Train_path)
all_data_Test = np.load(test_data_Test_path)
all_label_Test = np.load(label_data_Test_path)

mean_accuracy = []
mean_precision = []
mean_recall = []
mean_F1 = []

test_index = list(range(np.shape(test_data_Test)[0]))
train_index = list(range(190))

train_data = all_data
train_label = all_label

index_train_0 = np.where(train_label == 0)[0]
index_train_1 = np.where(train_label == 1)[0]
"""
  feature 7, Hausdorff distance
"""
feature_7_hausdorff_dist_all_0 = np.zeros((190, 2))
feature_7_hausdorff_dist_all_1 = np.zeros((190, 2))
feature_7_hausdorff_dist_all_0_Test = np.zeros((np.shape(test_data_Test)[0], 2))
feature_7_hausdorff_dist_all_1_Test = np.zeros((np.shape(test_data_Test)[0], 2))
# train
for mk in range(190):
    temp_0 = []
    temp_1 = []
    for jk in range(190):
        if jk == mk:
            continue
        else:
            if jk in index_train_0:
                temp_0.append(hausdorff_dist_matrix[train_index[jk], train_index[mk]])
            elif jk in index_train_1:
                temp_1.append(hausdorff_dist_matrix[train_index[jk], train_index[mk]])
    feature_7_hausdorff_dist_all_0[train_index[mk], 0] = sum(temp_0) / (np.shape(temp_0)[0])
    feature_7_hausdorff_dist_all_1[train_index[mk], 0] = sum(temp_1) / (np.shape(temp_1)[0])
    feature_7_hausdorff_dist_all_0[train_index[mk], 1] = np.std(temp_0) - np.std(temp_1)
    feature_7_hausdorff_dist_all_1[train_index[mk], 1] = np.std(temp_1)
# test
for mk in range(np.shape(test_data_Test)[0]):
    temp_0 = []
    temp_1 = []
    for jk in range(190):
        if jk in index_train_0:
            temp_0.append(hausdorff_dist_matrix_Test[train_index[jk], test_index[mk]])
        elif jk in index_train_1:
            temp_1.append(hausdorff_dist_matrix_Test[train_index[jk], test_index[mk]])
    feature_7_hausdorff_dist_all_0_Test[test_index[mk], 0] = sum(temp_0) / (np.shape(temp_0)[0])
    feature_7_hausdorff_dist_all_1_Test[test_index[mk], 0] = sum(temp_1) / (np.shape(temp_1)[0])
    feature_7_hausdorff_dist_all_0_Test[test_index[mk], 1] = np.std(temp_0) - np.std(temp_1)
    feature_7_hausdorff_dist_all_1_Test[test_index[mk], 1] = np.std(temp_1)
"""
  feature 7, Chamfer distance
"""
feature_7_chamfer_dist_all_0 = np.zeros((190, 2))
feature_7_chamfer_dist_all_1 = np.zeros((190, 2))
feature_7_chamfer_dist_all_0_Test = np.zeros((np.shape(test_data_Test)[0], 2))
feature_7_chamfer_dist_all_1_Test = np.zeros((np.shape(test_data_Test)[0], 2))
# train
for mk in range(190):
    temp_0 = []
    temp_1 = []
    for jk in range(190):
        if jk == mk:
            continue
        else:
            if jk in index_train_0:
                temp_0.append(chamfer_dist_matrix[train_index[jk], train_index[mk]])
            elif jk in index_train_1:
                temp_1.append(chamfer_dist_matrix[train_index[jk], train_index[mk]])
    feature_7_chamfer_dist_all_0[train_index[mk], 0] = sum(temp_0) / (np.shape(temp_0)[0])
    feature_7_chamfer_dist_all_1[train_index[mk], 0] = sum(temp_1) / (np.shape(temp_1)[0])
    feature_7_chamfer_dist_all_0[train_index[mk], 1] = np.std(temp_0) - np.std(temp_1)
    feature_7_chamfer_dist_all_1[train_index[mk], 1] = np.std(temp_1)
# test
for mk in range(np.shape(test_data_Test)[0]):
    temp_0 = []
    temp_1 = []
    for jk in range(190):
        if jk in index_train_0:
            temp_0.append(chamfer_dist_matrix_Test[train_index[jk], test_index[mk]])
        elif jk in index_train_1:
            temp_1.append(chamfer_dist_matrix_Test[train_index[jk], test_index[mk]])
    feature_7_chamfer_dist_all_0_Test[test_index[mk], 0] = sum(temp_0) / (np.shape(temp_0)[0])
    feature_7_chamfer_dist_all_1_Test[test_index[mk], 0] = sum(temp_1) / (np.shape(temp_1)[0])
    feature_7_chamfer_dist_all_0_Test[test_index[mk], 1] = np.std(temp_0) - np.std(temp_1)
    feature_7_chamfer_dist_all_1_Test[test_index[mk], 1] = np.std(temp_1)
"""
  feature 7, Wasserstein distance
"""
feature_7_Wasserstein_dist_all_0 = np.zeros((190, 2))
feature_7_Wasserstein_dist_all_1 = np.zeros((190, 2))
feature_7_Wasserstein_dist_all_0_Test = np.zeros((np.shape(test_data_Test)[0], 2))
feature_7_Wasserstein_dist_all_1_Test = np.zeros((np.shape(test_data_Test)[0], 2))
# train
for mk in range(190):
    temp_0 = []
    temp_1 = []
    for jk in range(190):
        if jk == mk:
            continue
        else:
            if jk in index_train_0:
                temp_0.append(Wasserstein_dist_matrix[train_index[jk], train_index[mk]])
            elif jk in index_train_1:
                temp_1.append(Wasserstein_dist_matrix[train_index[jk], train_index[mk]])
    feature_7_Wasserstein_dist_all_0[train_index[mk], 0] = sum(temp_0) / (np.shape(temp_0)[0])
    feature_7_Wasserstein_dist_all_1[train_index[mk], 0] = sum(temp_1) / (np.shape(temp_1)[0])
    feature_7_Wasserstein_dist_all_0[train_index[mk], 1] = np.std(temp_0) - np.std(temp_1)
    feature_7_Wasserstein_dist_all_1[train_index[mk], 1] = np.std(temp_1)
# test
for mk in range(np.shape(test_data_Test)[0]):
    temp_0 = []
    temp_1 = []
    for jk in range(190):
        if jk in index_train_0:
            temp_0.append(Wasserstein_dist_matrix_Test[train_index[jk], test_index[mk]])
        elif jk in index_train_1:
            temp_1.append(Wasserstein_dist_matrix_Test[train_index[jk], test_index[mk]])
    feature_7_Wasserstein_dist_all_0_Test[test_index[mk], 0] = sum(temp_0) / (np.shape(temp_0)[0])
    feature_7_Wasserstein_dist_all_1_Test[test_index[mk], 0] = sum(temp_1) / (np.shape(temp_1)[0])
    feature_7_Wasserstein_dist_all_0_Test[test_index[mk], 1] = np.std(temp_0) - np.std(temp_1)
    feature_7_Wasserstein_dist_all_1_Test[test_index[mk], 1] = np.std(temp_1)

# combine all features
feature_all[:, 23:25] = feature_7_hausdorff_dist_all_0[:, 0:2]
feature_all[:, 25:27] = feature_7_hausdorff_dist_all_1[:, 0:2]
feature_all[:, 27:29] = feature_7_chamfer_dist_all_0[:, 0:2]
feature_all[:, 29:31] = feature_7_chamfer_dist_all_1[:, 0:2]
feature_all[:, 31:33] = feature_7_Wasserstein_dist_all_0[:, 0:2]
feature_all[:, 33:35] = feature_7_Wasserstein_dist_all_1[:, 0:2]

# combine all features for test dataset
feature_all_Test[:, 23:25] = feature_7_hausdorff_dist_all_0_Test[:, 0:2]
feature_all_Test[:, 25:27] = feature_7_hausdorff_dist_all_1_Test[:, 0:2]
feature_all_Test[:, 27:29] = feature_7_chamfer_dist_all_0_Test[:, 0:2]
feature_all_Test[:, 29:31] = feature_7_chamfer_dist_all_1_Test[:, 0:2]
feature_all_Test[:, 31:33] = feature_7_Wasserstein_dist_all_0_Test[:, 0:2]
feature_all_Test[:, 33:35] = feature_7_Wasserstein_dist_all_1_Test[:, 0:2]

# normalize all the data
for i in range(23, 35):
    feature_all[:, i] = preprocessing.normalize([feature_all[:, i]])
# print(feature_all[:, 23:35])
# print(feature_all[:, 0:2])

# normalize all the data
for i in range(23, 35):
    feature_all_Test[:, i] = preprocessing.normalize([feature_all_Test[:, i]])

# feature_all[np.array(train_index)[:,np.newaxis], np.array(list([1, 7,  8, 11, 22, 28, 29, 32]))[:]]
# feature_all[np.array(train_index)[:,np.newaxis], np.array(list(range(35)))[:]]
# feature_all[np.array(train_index)[:,np.newaxis], np.array(list([7]))[:]]
# feature_all[np.array(train_index)[:,np.newaxis], np.array(list(range(0, 16)) + list(range(23, 35)))[:]]
# feature_all[np.array(train_index)[:,np.newaxis], np.array(list(range(0, 23)))[:]]
X_train = feature_all[np.array(train_index)[:, np.newaxis], np.array(list(range(35)))[:]]
y_train = all_label[train_index]
X_test = feature_all_Test[np.array(test_index)[:, np.newaxis], np.array(list(range(35)))[:]]
y_test = all_label_Test[test_index]

# balancing the class classification for the training data
sm = SMOTE(random_state=i, k_neighbors=1)  # set the seed of smote as the index of the loop
X_train, y_train = sm.fit_resample(X_train, y_train)
index_train_0_try = np.where(y_train == 0)[0]
index_train_1_try = np.where(y_train == 1)[0]
#   print(index_train_0_try)
#   print(index_train_1_try)

# print(np.shape(X_train))
# print(np.shape(y_train))

# # PCA
# pca = PCA(n_components = 10)
# X_train = pca.fit_transform(X_train)
# X_test = pca.transform(X_test)

# model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model = XGBClassifier(
    learning_rate=0.01,
    n_estimators=5000,
    max_depth=9,
    min_child_weight=1,
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=1,
    objective='binary:logistic',
    eval_metric='error@0.001',
    nthread=4,
    scale_pos_weight=1,
    seed=27)
model.fit(X_train, y_train)

# predict the results
y_pred = model.predict(X_test)
# y_pred = np.logical_not(y_pred).astype('int64')  # be careful, we flip 0 and 1 here
# y_test = np.logical_not(y_test).astype('int64')  # be careful, we flip 0 and 1 here
# print(classification_report(y_test,y_pred))
mean_accuracy.append(accuracy_score(y_test, y_pred))
mean_precision.append(precision_score(y_test, y_pred))
mean_recall.append(recall_score(y_test, y_pred))
mean_F1.append(f1_score(y_test, y_pred))
# print('accuracy:', accuracy_score(y_test, y_pred))
print('The confusion matrix is:')
print(confusion_matrix(y_test, y_pred))

# print(y_pred)
# print(y_test)
N = 1
print('The accuracy is:', sum(mean_accuracy)/N)
print('The precision is:', sum(mean_precision)/N)
print('The recall is:', sum(mean_recall)/N)
print('The F1-score is:', sum(mean_F1)/N)